In [1]:
import cv2 
from matplotlib import pyplot as plt
import numpy as np
import sklearn
import math # need math.sqrt() for Euclidean Alg

In [2]:
class Stack:
    def __init__(self):
        self.my_stack = [] # when intialized a list structure will be created to be used as a stack with local variable my_stack
        
    def push(self, value):
        self.my_stack.append(value) # pushing involves adding value to top of stack or in a linear sense - the end of the list
        
    def pop(self):
        return self.my_stack.pop() # using python's built in pop() function to remove last value added to list -> will return this value so that we can call it and compare it directly to seed
        
    def stack_size(self):
        return len(self.my_stack) # will be used in following function to determine when stack is empty -> value is returned so it can be compared to empty

    def print_stack(self):
        if len(self.my_stack) == 0:
            print("EMPTY")
        else:
            for i in range(len(self.my_stack)):
                print(self.my_stack[i])
                 
    def isNotEmpty(self):
        if len(self.my_stack) == 0:
            return False
        else:
            return True
    
    def clear(self):
        self.my_stack.clear()

In [3]:
def trackChanged(x): # empty pointer function for getTrackbar pos
    pass

def getIntensityValue():
    return cv2.getTrackbarPos('Intensity', 'FloodFill') # returns value of "Intensity" based on trackbar position

def checkIfVisited(coord): # checking if pixel currently being evaluated has already been checked before - this will prevent infinite loops 
    global visited_pixel
    count = 0
    for i in range(len(visited_pixel)): # for every item in visited list compare their row/col values
        if coord != visited_pixel[i]:
            count += 1
    if count == len(visited_pixel): # if for all values in list they did not match then it HAS NOT been visited already
        return True
    else:
        return False
    
def checkInRange(coord):
    global img_width, img_height
    if coord[0] < 0 or coord[0] > img_width-1  or coord[1] < 0 or coord[1] > img_height-1:
        return False
    else:
        return True
    

In [4]:
# After multiple attempts and implementation of different algorithms I decided that Euclideans Algorithm would be the best fit for this problem
# Euclidean ALG: 
#  - Will be summing the linear distance between 2 points (seend and current) for each of the RGB dimensions
#  - This when compared to associated threshold will be able to compare two points in a uniform space and evaluate how close in characteristics they are (in this case R/G/B)
def compareToSeed(pixel): # comparing seed_pixel and current_Pixel being evaluated to see if its RGB values are within intensity threshold
    global cpy, seed_pixel, the_stack, to_be_filled, current_pixel
    threshold = getIntensityValue()
    R = (int(cpy[current_pixel[0],current_pixel[1]][2]) - int(cpy[pixel[0], pixel[1]][2]))**2 # BGR image
    G = (int(cpy[current_pixel[0],current_pixel[1]][1]) - int(cpy[pixel[0], pixel[1]][1]))**2 
    B = (int(cpy[current_pixel[0],current_pixel[1]][0]) - int(cpy[pixel[0], pixel[1]][0]))**2
    delta_E = math.sqrt(R+G+B)
    
    #print("Delta_E: ", delta_E)
    #print("Threshold: ", threshold)
    
    if delta_E <= threshold:
        the_stack.push(pixel)
        # print("Pushed: ", pixel)
        to_be_filled.append(pixel)

In [5]:
# event = what is happening on the image
# x,y = location of mouse during event
# flags = specific condition when a mouse event occurs
# param = userdata that is passed when we call back floodfill function

def floodFill(event, x, y, flags, param):
    global cpy, seed_pixel, the_stack, current_pixel, to_be_filled, visited_pixel
    if event == cv2.EVENT_LBUTTONDOWN:
        seed_pixel = (y, x)
        the_stack.push(seed_pixel) # this will push the col, row values onto the stack as initial val --> col, row is actually x/y position
        
        while the_stack.stack_size() > 0:
            current_pixel = the_stack.pop() # the current_pixel x/y is always getting updated by whatever neighbours meet criteria
            neighbours = [(current_pixel[0]-1, current_pixel[1]),(current_pixel[0]+1, current_pixel[1]), (current_pixel[0], current_pixel[1]+1),
                          (current_pixel[0], current_pixel[1]-1)] # getting NSEW pixels aka 4-CONNECTIVITY
            #print("Current Pixel: ", current_pixel)
            # print("Neighbours ---> N_pixel: {}, S_pixel: {}, E_pixel: {}, W_pixel: {}".format(neighbours[0], neighbours[1], neighbours[2], neighbours[3]))
            
            for i in range(4):
                if checkIfVisited(neighbours[i]): # if it hasnt been already visited
                    if checkInRange(neighbours[i]): # and if it is within range
                        compareToSeed(neighbours[i]) # compare it 
                        visited_pixel.append(neighbours[i]) # and add it to already visited list
                # print("Stack Size: ", the_stack.stack_size())

        for j in range(len(to_be_filled)): # fill the pixels of BGR image with B value
            cpy[to_be_filled[j][0], to_be_filled[j][1]] = [255, 0, 0] # converting y,x back to xy
            if j == len(to_be_filled) - 1:
                print("DONE")
    cv2.imshow("FloodFill", cpy)

In [7]:
baboon = cv2.imread("baboon1.png")
queens_flag = cv2.imread("queensflag.png")
while True: 
    x = input("Please Enter Which Image You Would Like To Test --> (1) = Baboon ... (2) = Flag: ")
    if int(x) == 1:
        cpy = baboon.copy()
        img_width, img_height, img_channel = baboon.shape # initializing array dimensions for looping purposes
        print("BEST THRESHOLD VALUE IS 0-30")
        break
    elif int(x) == 2:
        cpy = queens_flag.copy()
        img_width, img_height, img_channel = queens_flag.shape # initializing array dimensions for looping purposes
        print("EXPECT LONG COMPUTATION DUE TO LARGE UNIFORM AREAS - REQUIRES A LOT OF TIME FOR COMPUTATIONS (5-10min)") 
        break
    else:
        print("ERROR - Try again")
        continue 
      
the_stack = Stack() # initializing a global stack variable based of class definition
visited_pixel = []
current_pixel = [0, 0, 0]
seed_pixel = [0, 0, 0]
to_be_filled = []

cv2.namedWindow("FloodFill", cv2.WINDOW_NORMAL)
cv2.createTrackbar('Intensity', "FloodFill", 0, 30, trackChanged)# imlementing trackbar function --> this will create a variable intensity threshold to which the homogenity criterion will be based off of
cv2.setMouseCallback("FloodFill", floodFill)
while True:
    cv2.imshow("FloodFill", cpy)
    if cv2.waitKey(1) == 27:
            break 
cv2.destroyAllWindows()

Please Enter Which Image You Would Like To Test --> (1) = Baboon ... (2) = Flag:  1


BEST THRESHOLD VALUE IS 0-30
gg
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
